In [ ]:
val username = ""
val password = ""

In [32]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.{SQLContext, DataFrame}
import org.apache.spark.sql.functions._

val conf = new SparkConf()
                            .setAppName("lab3")
                            .setAll(Map("spark.cassandra.connection.host" -> "10.0.1.9",
                                        "spark.cassandra.connection.port" -> "9042",
                                        "spark.cassandra.auth.username" -> username,
                                        "spark.cassandra.auth.password" -> password))

val sparkSession = SparkSession.builder()
                                  .config(conf=conf)
                                  .getOrCreate()

var sc = sparkSession.sparkContext
val sqlContext = new SQLContext(sc)

conf = org.apache.spark.SparkConf@75f3e746
sparkSession = org.apache.spark.sql.SparkSession@303fb1df
sc = org.apache.spark.SparkContext@371b6748
sqlContext = org.apache.spark.sql.SQLContext@44ce6f65


org.apache.spark.sql.SQLContext@44ce6f65

###### драйвер: https://mvnrepository.com/artifact/org.postgresql/postgresql/42.2.12

In [2]:
%AddJar file:///data/home/kirill.likhouzov/Drivers/postgresql-42.2.12.jar
%AddJar file:///data/home/kirill.likhouzov/Drivers/spark-cassandra-connector_2.11-2.4.3.jar
%AddJar file:///data/home/kirill.likhouzov/Drivers/elasticsearch-spark-20_2.11-7.6.2.jar

Starting download from file:///data/home/kirill.likhouzov/Drivers/postgresql-42.2.12.jar


Waiting for a Spark session to start...

Finished download of postgresql-42.2.12.jar
Starting download from file:///data/home/kirill.likhouzov/Drivers/spark-cassandra-connector_2.11-2.4.3.jar
Finished download of spark-cassandra-connector_2.11-2.4.3.jar
Starting download from file:///data/home/kirill.likhouzov/Drivers/elasticsearch-spark-20_2.11-7.6.2.jar
Finished download of elasticsearch-spark-20_2.11-7.6.2.jar


## Input Data

### hdfs

In [3]:
val weblogs = spark
                .read
                .parquet("/labs/laba03/weblogs.parquet")
                .repartition(1)
                .cache()

weblogs = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [4]:
weblogs.count()

36138

In [5]:
weblogs.show(1, 1000, true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### postgress

In [6]:
val domain_cats = spark
                    .read
                    .format("jdbc")
                    .option("url", "jdbc:postgresql://10.0.1.9:5432/labdata")
                    .option("dbtable", "domain_cats")
                    .option("user", username)
                    .option("password", password)
                    .option("driver", "org.postgresql.Driver")
                    .load()
                    .repartition(1)
                    .cache()

domain_cats = [domain: string, category: string]


[domain: string, category: string]

In [7]:
domain_cats.count()

245981

In [8]:
domain_cats.show(1, false)

+----------------------+--------+
|domain                |category|
+----------------------+--------+
|national-lottery.co.uk|gambling|
+----------------------+--------+
only showing top 1 row



### ElasticSearch

In [9]:
val visits = spark
            .read
            .format("org.elasticsearch.spark.sql")
            .option("es.nodes.wan.only","true")
            .option("es.nodes", "10.0.1.9")
            .option("es.port", "9200")
            .load("visits")
            .repartition(1)
            .cache()

visits = [category: string, event_type: string ... 4 more fields]


[category: string, event_type: string ... 4 more fields]

In [10]:
visits.count()

182540

In [11]:
visits.show(1, false)

+-----------------------+----------+-------------------------+----------+-------------------+------------------------------------+
|category               |event_type|item_id                  |item_price|timestamp          |uid                                 |
+-----------------------+----------+-------------------------+----------+-------------------+------------------------------------+
|Entertainment-equipment|buy       |Entertainment-equipment-2|2529      |2020-01-01 11:00:00|40b29579-e845-45c0-a34d-03630d296a81|
+-----------------------+----------+-------------------------+----------+-------------------+------------------------------------+
only showing top 1 row



### Cassandra

In [12]:
import org.apache.spark.sql.cassandra._
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql.CassandraConnector

val connector = CassandraConnector(sc.getConf)

connector = com.datastax.spark.connector.cql.CassandraConnector@32c3f3b


com.datastax.spark.connector.cql.CassandraConnector@32c3f3b

In [13]:
val clients= spark.read
                .format("org.apache.spark.sql.cassandra")
                .options(Map("table" -> "clients", "keyspace" -> "labdata"))
                .load()
                .repartition(1)
                .cache()

clients = [uid: string, age: int ... 1 more field]


[uid: string, age: int ... 1 more field]

In [14]:
clients.count()

36138

In [15]:
clients.show(1, false)

+------------------------------------+---+------+
|uid                                 |age|gender|
+------------------------------------+---+------+
|2f3a4f67-2790-45b4-bdd2-7fd6d3e7c24b|39 |M     |
+------------------------------------+---+------+
only showing top 1 row



## Aggregation

### visits

In [16]:
val clients_ages = clients.withColumn("age_category",  when($"age" >= 18 && $"age" <= 24, "18-24")
                                                      .when($"age" >= 25 && $"age" <= 34, "25-34")
                                                      .when($"age" >= 35 && $"age" <= 44, "35-44")
                                                      .when($"age" >= 45 && $"age" <= 54, "45-54")
                                                      .when($"age" >= 55, ">=55")
                                                      .otherwise(""))
clients_ages.show(1, false)

+------------------------------------+---+------+------------+
|uid                                 |age|gender|age_category|
+------------------------------------+---+------+------------+
|2f3a4f67-2790-45b4-bdd2-7fd6d3e7c24b|39 |M     |35-44       |
+------------------------------------+---+------+------------+
only showing top 1 row



clients_ages = [uid: string, age: int ... 2 more fields]


error: error while loading QualifiedTableName, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location not matching its contents: contains 

[uid: string, age: int ... 2 more fields]

In [17]:
val visits_cl = clients_ages.as("a")
                    .join(visits.as("b"), $"a.uid" === $"b.uid", "inner")
                    .select($"a.uid"
                            ,$"a.gender"
                            ,$"a.age_category"
                            ,concat(lit("shop_"), lower(regexp_replace($"b.category", "-", "_"))).as("web_category")
                            ,$"b.timestamp")
                    .filter(!$"web_category".isNull)
                    .cache()

visits_cl.show(1, false)

+------------------------------------+------+------------+----------------------------+-------------------+
|uid                                 |gender|age_category|web_category                |timestamp          |
+------------------------------------+------+------------+----------------------------+-------------------+
|40b29579-e845-45c0-a34d-03630d296a81|F     |45-54       |shop_entertainment_equipment|2020-01-01 11:00:00|
+------------------------------------+------+------------+----------------------------+-------------------+
only showing top 1 row



visits_cl = [uid: string, gender: string ... 3 more fields]


[uid: string, gender: string ... 3 more fields]

In [18]:
val visits_web = 
    visits_cl
        .groupBy("uid")
        .pivot("web_category")
        .count()
        
visits_web.filter($"uid" === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").show(1, 100, true)

-RECORD 0--------------------------------------------------------------
 uid                            | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777 
 shop_cameras                   | null                                 
 shop_clothing                  | null                                 
 shop_computers                 | 2                                    
 shop_cosmetics                 | null                                 
 shop_entertainment_equipment   | 1                                    
 shop_everyday_jewelry          | null                                 
 shop_house_repairs_paint_tools | null                                 
 shop_household_appliances      | null                                 
 shop_household_furniture       | null                                 
 shop_kitchen_appliances        | null                                 
 shop_kitchen_utensils          | null                                 
 shop_luggage                   | null                          

visits_web = [uid: string, shop_cameras: bigint ... 15 more fields]


[uid: string, shop_cameras: bigint ... 15 more fields]

### weblogs

In [19]:
val weblogs_explode = weblogs
                            .select($"uid"
                                    ,explode($"visits").as("web"))
                            .cache()

weblogs_explode

weblogs_explode = [uid: string, web: struct<timestamp: bigint, url: string>]


[uid: string, web: struct<timestamp: bigint, url: string>]

In [20]:
val weblogs_url = weblogs_explode
                            .withColumn("url", weblogs_explode("web.url"))
                            .withColumn("timestamp", weblogs_explode("web.timestamp"))
                            .withColumn("host", callUDF("parse_url", $"url", lit("HOST")))
                            .select($"uid", $"url", $"host", $"timestamp")
                            .cache()

weblogs_url.show(1, 1000, true)

-RECORD 0----------------------------------------------
 uid       | 3761db96-4a89-4612-be74-9567ab2f21aa      
 url       | http://na.buhgalteria.ru/document/n117802 
 host      | na.buhgalteria.ru                         
 timestamp | 1419596441719                             
only showing top 1 row



weblogs_url = [uid: string, url: string ... 2 more fields]


[uid: string, url: string ... 2 more fields]

In [21]:
val weblogs_cl = clients_ages.as("a")
                    .join(weblogs_url.as("b"), $"a.uid" === $"b.uid", "inner")
                    .select($"a.uid"
                            ,$"a.gender"
                            ,$"a.age_category"
                            ,$"b.host"
                            ,$"b.timestamp")
                    .cache()

weblogs_cl.show(1, false)

+------------------------------------+------+------------+-----------------+-------------+
|uid                                 |gender|age_category|host             |timestamp    |
+------------------------------------+------+------------+-----------------+-------------+
|3761db96-4a89-4612-be74-9567ab2f21aa|F     |35-44       |na.buhgalteria.ru|1419596441719|
+------------------------------------+------+------------+-----------------+-------------+
only showing top 1 row



weblogs_cl = [uid: string, gender: string ... 3 more fields]


[uid: string, gender: string ... 3 more fields]

In [22]:
val weblogs_cl_cat = weblogs_cl.as("a")
                        .join(domain_cats.as("b"), regexp_replace($"a.host", "www.", "") === $"b.domain", "left")
                        .select($"a.uid"
                                ,$"a.gender"
                                ,$"a.age_category"
                                ,$"a.host"
                                ,regexp_replace($"a.host", "www.", "").as("host_not_www")
                                ,$"a.timestamp"
                                ,concat(lit("web_"), lower(regexp_replace($"b.category", "-", "_"))).as("shop_category"))
                                .filter(!$"shop_category".isNull)
                        .cache()

weblogs_cl_cat.show(1, false)

+------------------------------------+------+------------+------------------+--------------+-------------+-------------+
|uid                                 |gender|age_category|host              |host_not_www  |timestamp    |shop_category|
+------------------------------------+------+------------+------------------+--------------+-------------+-------------+
|3761db96-4a89-4612-be74-9567ab2f21aa|F     |35-44       |www.buhgalteria.ru|buhgalteria.ru|1419596379453|web_finance  |
+------------------------------------+------+------------+------------------+--------------+-------------+-------------+
only showing top 1 row



weblogs_cl_cat = [uid: string, gender: string ... 5 more fields]


[uid: string, gender: string ... 5 more fields]

In [23]:
val weblogs_web = 
    weblogs_cl_cat
        .groupBy("uid")
        .pivot("shop_category")
        .agg(count('host_not_www))

//weblogs_web.show(1, 100, true)

weblogs_web = [uid: string, web_arts_and_entertainment: bigint ... 22 more fields]


[uid: string, web_arts_and_entertainment: bigint ... 22 more fields]

In [24]:
clients_ages.select($"uid").distinct().count() == clients_ages.count()

true

In [25]:
val res = clients_ages.as("a")
                        .join(visits_web.as("b"), $"a.uid" === $"b.uid", "left")
                        .join(weblogs_web.as("c"), $"a.uid" === $"c.uid", "left")
                        .select($"a.uid"
                                ,$"a.gender"
                                ,$"a.age_category".as("age_cat")
                                ,$"b.*"
                                ,$"c.*")
                        .drop($"b.uid")
                        .drop($"c.uid")
                        .cache()
                                
res.show(1, 1000, true)

-RECORD 0--------------------------------------------------------------
 uid                            | 0392f398-ea7e-4a1b-8c9d-d04df2a2a331 
 gender                         | F                                    
 age_cat                        | >=55                                 
 shop_cameras                   | 1                                    
 shop_clothing                  | null                                 
 shop_computers                 | 1                                    
 shop_cosmetics                 | null                                 
 shop_entertainment_equipment   | 1                                    
 shop_everyday_jewelry          | null                                 
 shop_house_repairs_paint_tools | null                                 
 shop_household_appliances      | null                                 
 shop_household_furniture       | null                                 
 shop_kitchen_appliances        | null                          

res = [uid: string, gender: string ... 40 more fields]


[uid: string, gender: string ... 40 more fields]

In [27]:
res.count()

36138

In [29]:
res
    .write
    .mode("overwrite")
    .format("jdbc")
    .option("url", s"jdbc:postgresql://10.0.1.9:5432/$username")
    .option("dbtable", "clients")
    .option("user", username)
    .option("password", password)
    .option("driver", "org.postgresql.Driver")
    .save()

In [30]:
//check
res.filter($"uid" === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").show(3, 1000, true)

-RECORD 0--------------------------------------------------------------
 uid                            | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777 
 gender                         | F                                    
 age_cat                        | 18-24                                
 shop_cameras                   | null                                 
 shop_clothing                  | null                                 
 shop_computers                 | 2                                    
 shop_cosmetics                 | null                                 
 shop_entertainment_equipment   | 1                                    
 shop_everyday_jewelry          | null                                 
 shop_house_repairs_paint_tools | null                                 
 shop_household_appliances      | null                                 
 shop_household_furniture       | null                                 
 shop_kitchen_appliances        | null                          

In [33]:
sc.stop()

In [ ]:
//GRANT SELECT ON TABLE clients TO PUBLIC;